# AES encryption-decryption

In [12]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

def generate_key(key_length):
    return get_random_bytes(key_length // 8)

def write_key_to_file(filename, key):
    with open(filename, 'wb') as file:
        file.write(key)

def read_key_from_file(filename):
    with open(filename, 'rb') as file:
        return file.read()

def pad(plaintext):
    block_size = AES.block_size
    padding_length = block_size - len(plaintext) % block_size
    padding = bytes([padding_length] * padding_length)
    return plaintext + padding

def unpad(padded_text):
    padding_length = padded_text[-1]
    return padded_text[:-padding_length]

def aes_encrypt(key, plaintext, use_cfb=False):
    cipher = AES.new(key, AES.MODE_CFB if use_cfb else AES.MODE_ECB)
    return cipher.encrypt(plaintext)

def aes_decrypt(key, ciphertext, use_cfb=False):
    cipher = AES.new(key, AES.MODE_CFB if use_cfb else AES.MODE_ECB)
    return cipher.decrypt(ciphertext)

In [13]:
import os
def aes():
    key_length = 256
    key_filename = 'aes_key.bin'
    plaintext_filename = 'text.txt'
    encrypted_filename = 'encrypted.bin'

    # Generate and store the key if it doesn't exist
    if not os.path.exists(key_filename):
        key = generate_key(key_length)
        write_key_to_file(key_filename, key)
    else:
        key = read_key_from_file(key_filename)

    # Read plaintext from a file
    with open(plaintext_filename, 'rb') as file:
        plaintext = file.read()

    # Pad the plaintext to a multiple of AES block size
    padded_plaintext = pad(plaintext)

    # Encrypt the data and write to a file
    encrypted_data = aes_encrypt(key, padded_plaintext, use_cfb=False)
    with open(encrypted_filename, 'wb') as file:
        file.write(encrypted_data)

    # Decrypt the data
    decrypted_data = aes_decrypt(key, encrypted_data, use_cfb=False)

    # Unpad the decrypted data
    unpadded_decrypted_data = unpad(decrypted_data)

    # Display the decrypted data
    print("Decrypted Data:")
    print(unpadded_decrypted_data.decode('utf-8'))

In [14]:
aes()

Decrypted Data:
﻿Our concept of time begins with the creation of the universe.
Therefore if the laws of nature created the universe, these laws must have existed prior to time; that is the laws of nature would be outside of time.
What we have then is totally non-physical laws, outside of time, creating a universe. Now that description might sound somewhat familiar.
Very much like the biblical concept of God: not physical, outside of time, able to create a universe.


# RSA signature

### Here is the [tutorial link](https://cryptobook.nakov.com/digital-signatures/rsa-sign-verify-examples)

In [56]:
from Crypto.PublicKey import RSA
import pickle
import os

key_filename = 'rsa_key_object.bin'
plaintext_filename = 'text.txt'
signature_file = "signature.bin"

if not os.path.exists(key_filename):
    keyPair = RSA.generate(bits=1024)
    key = {
        "n": keyPair.n,
        "e": keyPair.e,
        "d": keyPair.d,
    }
    with open(key_filename, 'wb') as file:
        pickle.dump(key, file)
else:
    with open(key_filename, 'rb') as file:
        key = pickle.load(file)

In [57]:
from hashlib import sha512

with open(plaintext_filename, 'rb') as file:
    plaintext = file.read()
hash = int.from_bytes(sha512(plaintext).digest(), byteorder='big')
signature = pow(hash, key["d"], key["n"])

with open(signature_file, 'wb') as file:
    pickle.dump(signature, file)
        

In [58]:

with open(plaintext_filename, 'rb') as file:
    plaintext = file.read()
with open(signature_file, 'rb') as file:
    signature = pickle.load(file)
hash = int.from_bytes(sha512(plaintext).digest(), byteorder='big')
hashFromSignature = pow(signature, key["e"], key["n"])
print("Signature valid:", hash == hashFromSignature)
    

Signature valid: True


# RSA encryption-decryption

### Here is the [tutorial link](https://cryptobook.nakov.com/asymmetric-key-ciphers/rsa-encrypt-decrypt-examples)

In [59]:
from Crypto.Cipher import PKCS1_OAEP
import binascii

plaintext = b"hello"
    
encryptor = PKCS1_OAEP.new(keyPair.publickey())
encrypted = encryptor.encrypt(plaintext)
print("Encrypted:", binascii.hexlify(encrypted))

Encrypted: b'864ce42d7830d31959b04fcd1204edfd022091353bf68767f5a7bfff93b3cabb3c920dbf0fcbd09dfc8bcd10a8f6de006203838864ebb60fc4017223aaf65c7b6d00524c7a187c02a968c5884a9262a364e9e472b468e124ae4e0bea59c44c24c07125128df8663afed2c93c1534d2f8cc6922e6905d6fc5fc56d11a8214d456'


In [60]:
decryptor = PKCS1_OAEP.new(keyPair)
decrypted = decryptor.decrypt(encrypted)
print('Decrypted:', decrypted)

Decrypted: b'hello'
